In [ ]:
#1 - Import libraries
import requests
import pandas as pd
import time

In [ ]:
#2 - Keys
API_KEY = ""
#CHANNEL_ID = "UCX6OQ3DkcsbYNE6H8uQQuVA"             # Mr Beast
CHANNEL_ID = "UC-lHJZR3Gqxm24_Vd_AJ5Yw"             # PewDiePie
#CHANNEL_ID = "UCINb0wqPz-A0dV9nARjJlOQ"             # The Dodo

In [ ]:
#3 - Calling API
pageToken = ""

#from video
url = "https://www.googleapis.com/youtube/v3/search?key=" + API_KEY + "&channelId=" + CHANNEL_ID + "&part=snippet,id&order=date&maxResults=10000&" + pageToken
response = requests.get(url).json()
#response           # uncomment to see raw json file

In [ ]:
#4 - Display first item (latest upload)
response['items'][0]

In [ ]:
# Refer to #4 (grab value of videoId from key 'items')
# Comment out (optional)
video_id = response['items'][0]['id']['videoId']
video_id

In [ ]:
# Refer to #4 (grab value of title from key 'items')
# Remove excess characters with replace
# Comment out (optional)
video_title = response['items'][0]['snippet']['title']
video_title = str(video_title).replace("&amp;","")
video_title

In [ ]:
# Refer to #4 (grab value of publishedAt from key 'items')
# Remove all characters to the right after specified split character
# Comment out (optional)
upload_date = response['items'][0]['snippet']['publishedAt']
upload_date = str(upload_date).split("T")[0]
upload_date

In [ ]:
# 2nd API Call / video_id is called in video for loop
def get_video_information(video_id):
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id=" + video_id + "&part=statistics&key=" + API_KEY

    response_video_stats = requests.get(url_video_stats).json()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    #dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count, comment_count

In [ ]:
# Passing Pandas dataframe into function
def get_videos(df):
    # These 3 lines are from Cell # 3
    pageToken = ""
    url = "https://www.googleapis.com/youtube/v3/search?key=" + API_KEY + "&channelId=" + CHANNEL_ID + "&part=snippet,id&order=date&maxResults=10000&" + pageToken
    response = requests.get(url).json()

    time.sleep(1)       # 1 second delay / provides API enough time to process

    # For loop that grabs info specifically from videos
    for video in response['items']:
        if video['id']['kind'] == "youtube#video":
            video_id = video['id']['videoId']
            video_title = video['snippet']['title']
            video_title = str(video_title).replace("&amp;","")
            upload_date = video['snippet']['publishedAt']
            upload_date = str(upload_date).split("T")[0]

            # 2nd API call gets called here
            view_count, like_count, comment_count = get_video_information(video_id)


            # Saving data into Pandas dataframe with append
            df = df.append({
                'video_id': video_id,
                'video_title': video_title,
                'upload_date': upload_date,
                'view_count': view_count,
                'like_count': like_count,
                'comment_count': comment_count}, ignore_index=True)


            # Outputs
            # print(video_id)
            # print(video_title)
            # print(upload_date)
            # print(view_count)
            # print(like_count)
            # print(comment_count)

    return df

In [ ]:
# Build dataframe (built empty / populated after for loop)
df = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","comment_count"])

df = get_videos(df)